In [26]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen
from tqdm import tqdm_notebook


In [27]:
df_debut = pd.read_csv("../data/debut_album_1118.csv")

In [28]:
df_debut.tail(20)

,artist,album,genre,single_count
1929,Terror Universal,Make Them Bleed,Heavy metal,0
1930,NCT,NCT 2018 Empathy,"K-pop, hip hop, future bass, R&B, electropop",6
1931,Calum Scott,Only Human,Pop,3
1932,Kasbo,Places We Don't Know,"Electronic, future bass, house",6
1933,Mike Shinoda,Post Traumatic,Rap,1
1934,Maxo Kream,Punken,Trap,0
1935,Rich Homie Quan,Rich as in Spirit,Hip hop,2
1936,Shame,Songs of Praise,"Post-punk, indie rock",0
1937,Anne-Marie,Speak Your Mind,R&B,5
1938,Ã„ngie,Suicidal Since 1995,Pop,2


In [32]:
def get_genius(data):
    
    df_articles = pd.DataFrame(columns = ['artist', 'album', 'genius_frequency'])
    
    for i in tqdm_notebook(range(len(list(df_debut['artist'])))):

        artist = df_debut['artist'][i]
        album = df_debut['album'][i]
        
        artist_ = artist.replace(" ", "+")
        album_ = album.replace(" ", "+")
        
        page_num = 1
        articles = 0

        while True:

            url_ = 'https://genius.com/api/search/article?page=' + str(page_num) + '&q=' + artist_ + "+" + album_
            response = requests.get(url_).json()

            articles += len(response['response']['sections'][0]['hits'])

            page_num += 1

            if response['response']['next_page'] == None :
                break

        data = {
            'artist': artist,
            'album' : album,
            'genius_frequency' : articles
        }

#         print(data)

        df_articles.loc[len(df_articles)] = data
    
    return df_articles

In [33]:
genius_articles = get_genius(df_debut)

HBox(children=(IntProgress(value=0, max=1949), HTML(value='')))

In [34]:
print(genius_articles.shape)
genius_articles.tail()

(1949, 3)


,artist,album,genius_frequency
1944,EDEN,vertigo,0
1945,Nipsey Hussle,Victory Lap,1
1946,Rich the Kid,The World Is Yours,1
1947,Myles Kennedy,Year of the Tiger,0
1948,Fickle Friends,You Are Someone Else,0


In [46]:
genius_articles.loc[528, 'artist'] = 'Eve Ai'
genius_articles.loc[528, 'album'] = 'If You Love Me'
genius_articles.loc[1142, 'artist'] = 'Milica Pavlovia'
genius_articles.loc[1142, 'album'] = 'Gover Tela Body Language'
genius_articles.loc[1409, 'artist'] = 'Various Artists'
genius_articles.loc[1409, 'album'] = 'Fresh Filter : Volumn 1'

In [47]:
genius_articles.to_csv("../data/buzz_genius.csv", index=False, encoding='UTF-8')

In [48]:
df = pd.read_csv("../data/buzz_genius.csv")

In [49]:
df.shape

(1949, 3)

### Upload to MySQL database

In [50]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [51]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [52]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [53]:
df.to_sql(name="buzz_genius", con=engine, if_exists='replace')